# Caso: Análisis de CDR de una Empresa de Telecomunicaciones

## Cargar archivos csv
Utilizamos un Call Detail Record de ejemplo
##### Clientes llamantes: 199
##### Clientes llamados: 400
##### Horizonte de tiempo: De Nov-2012 a Ene-2014 (15 meses)

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
import time

In [ ]:
h = pd.read_csv('llamadas.csv')

Revisamos la información del DataFrame

In [ ]:
h.info()

Transformamos el DataFrame en un Grafo dirigido con pesos (cantidad de llamadas y duración)

In [ ]:
G = nx.from_pandas_edgelist(h, source="fromuserid", target="touserid", edge_attr=["numbercalls","secondscalls"], create_using=nx.DiGraph())
print(nx.info(G))

Crear la función top_nodes que mostrará los valores más altos de un diccionario

In [ ]:
def get_top_nodes(cdict, num=5):
    top_nodes ={}
    for i in range(num):
        top_nodes =dict(
            sorted(cdict.items(), key=lambda x: x[1], reverse=True)[:num]
            )
        return top_nodes

#### Grado

Guardar el grado de cada nodo en un diccionario

In [ ]:
gdeg=G.degree()

In [ ]:
G.degree(350024,weight="secondscalls")

In [ ]:
get_top_nodes(dict(gdeg))

In [ ]:
plt.hist(nx.degree_histogram(G))

In [ ]:
print(nx.info(G,350159))

In [ ]:
plt.figure(figsize=(80,45)) 
nx.draw_networkx(G, width=0.1, node_size=[100*val for(node,val)in gdeg])
plt.show()

#### In-Degree

In [ ]:
indeg=G.in_degree()
get_top_nodes(dict(indeg))

#### Out-Degree

In [ ]:
outdeg=G.out_degree()
get_top_nodes(dict(outdeg))

In [ ]:
nx.set_node_attributes(G,dict(gdeg), 'd')
nx.set_node_attributes(G,dict(indeg), 'id')
nx.set_node_attributes(G,dict(outdeg), 'od')

In [ ]:
gdeg=G.degree(weight="secondscalls")
indeg=G.in_degree(weight="secondscalls")
outdeg=G.out_degree(weight="secondscalls")
nx.set_node_attributes(G,dict(gdeg), 'wd')
nx.set_node_attributes(G,dict(indeg), 'wid')
nx.set_node_attributes(G,dict(outdeg), 'wod')

#### Degree Centrality

In [ ]:
degree_centrality =nx.degree_centrality(G)
nx.set_node_attributes(G,degree_centrality, 'dc')
get_top_nodes(degree_centrality)

In [ ]:
plt.figure(figsize=(80,45)) 
nx.draw_networkx(G, width=0.1, node_size=[1000*v for v in nx.get_node_attributes(G, 'dc').values()])

#### Betweenness

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G,weight="secondscalls")
nx.set_node_attributes(G,betweenness_centrality, 'bc')

In [ ]:
get_top_nodes(betweenness_centrality)

In [ ]:
plt.figure(figsize=(80,45)) 
nx.draw_networkx(G, width=0.1, node_size=[10000 * v for v in nx.get_node_attributes(G, 'bc').values()])

#### Closeness

In [ ]:
closeness_centrality =nx.closeness_centrality(G,distance="secondscalls")
nx.set_node_attributes(G,closeness_centrality, 'cc')

In [ ]:
get_top_nodes(closeness_centrality)

In [ ]:
plt.figure(figsize=(80,45)) 
pos =nx.spring_layout(G)
ec =nx.draw_networkx_edges(G, width=0.1, pos=pos)
nc =nx.draw_networkx_nodes(G, pos=pos,
                           node_color=[v for v in nx.get_node_attributes(G, 'cc').values()],
                           node_size=[100000 * v for v in nx.get_node_attributes(G, 'cc').values()])
lb =nx.draw_networkx_labels(G,pos =pos)

#### Eigenvector Centrality

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(G,weight="secondscalls")
nx.set_node_attributes(G, eigenvector_centrality,'ec')

In [ ]:
get_top_nodes(eigenvector_centrality)

In [ ]:
plt.figure(figsize=(80,45)) 
nx.draw_networkx(G, width=0.1, node_size=[20000 * v for v in nx.get_node_attributes(G, 'ec').values()])

#### PageRank Centrality

In [ ]:
pagerank_centrality =nx.pagerank(G,weight="secondscalls")
nx.set_node_attributes(G, pagerank_centrality, 'pr')

In [ ]:
get_top_nodes(pagerank_centrality)

In [ ]:
plt.figure(figsize=(80,45)) 
nx.draw_networkx(G, width=0.1, node_size=[50000 * v for v in nx.get_node_attributes(G, 'pr').values()], pos=pos)

## Métricas de Grafo

#### Average Path Length

In [ ]:
nx.average_shortest_path_length(G)

#### Diametro

In [ ]:
nx.diameter(G)

#### Densidad

In [ ]:
nx.density(G)

## Calculando el Valor de Red para cada Cliente

Extraer las métricas por nodo a un dataframe de Pandas

In [ ]:
graph = G.nodes(data = True)

In [ ]:
def func(graph):
    data={}
    data['fromuserid']=[x[0] for x in graph]
    other_cols = list([*graph][0][1].keys())
    for key in other_cols:
        data[key] = [x[1][key] for x in graph]
    return data

In [ ]:
df = pd.DataFrame(func(graph)).sort_values(by=['fromuserid']).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.info()

Exportando para utilizar fuera

In [ ]:
df.to_csv(r'../data/ind_grafo.csv',index=False)

Elaborado por Luis Cajachahua bajo licencia MIT (2021)